In [4]:
import tensorflow as tf
import math
import numpy as np
import scipy
import h5py
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization  
from keras.layers import Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from keras.applications.imagenet_utils import preprocess_input
from matplotlib.pyplot import imshow

In [7]:
# identity block 
def identity_block(x,f,filters):

  # extracting number of filters for each block. each block has 3 conv 
  f1,f2,f3 = filters

  #input value stored in shortcut
  x_shortcut = x

  # first conv 
  x = Conv2D(filters=f1,kernel_size=(1,1),strides=(1,1),padding='valid', kernel_initializer=glorot_uniform(seed=0))(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  # second conv
  x = Conv2D(filters=f2,kernel_size=(f,f),strides=(1,1),padding='valid', kernel_initializer=glorot_uniform(seed=0))(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  #third conv
  x = Conv2D(filters=f3,kernel_size=(1,1),strides=(1,1),padding='valid', kernel_initializer=glorot_uniform(seed=0))(x)
  x = BatchNormalization(axis=3)(x)


  #adding the shortcut 
  x = Add[(x,x_shortcut)]
  x = Activation('relu')(x)

  #return the output tensor
  return x




In [8]:
# the convolutional block
def con_block(x,f,filters,s=2):

  # filters 
  f1,f2,f3 = filters

  x_copy = x 

   # first conv 
  x = Conv2D(filters=f1,kernel_size=(1,1),strides=(s,s),padding='valid', kernel_initializer=glorot_uniform(seed=0))(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  # second conv
  x = Conv2D(filters=f2,kernel_size=(f,f),strides=(1,1),padding='valid', kernel_initializer=glorot_uniform(seed=0))(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  #third conv
  x = Conv2D(filters=f3,kernel_size=(1,1),strides=(1,1),padding='valid', kernel_initializer=glorot_uniform(seed=0))(x)
  x = BatchNormalization(axis=3)(x)

  #creating the con_shortcut
  x_copy = Conv2D(filters=f3, kernel_size=(1,1),strides=(s,s),padding='valid', kernel_initializer=glorot_uniform(seed=0))(x_copy)
  x_copy = BatchNormalization(axis=3)(x_copy)

  # adding them
  x = Add[(x,x_copy)]
  x = Activation('relu')

  return x






In [2]:
# defining the resnet 101
def resnet101(input_shape,output_classes):

  x_input = input(input_shape)
  x = ZeroPadding2D((3, 3))(x_input)

  # Stage 1
  x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(x)
  x = BatchNormalization(axis=3, name='bn_conv1')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  #stage 2 where layers begin
  x = con_block(x,3,[64,64,256],s=1)
  x = identity_block(x,3,[64,64,256])
  x = identity_block(x,3,[64,64,256])

  # layer 2
  x = con_block(x,3,[128,128,512],s=2)
  x = identity_block(x,3,[128,128,512])
  x = identity_block(x,3,[128,128,512])
  x = identity_block(x,3,[128,128,512])

  #layer 
  x = con_block(x,3,[256,256,1024],s=2) 
  for i in range(1,23):
    x = identity_block(x,3,[256,256,1024])     

  #layer 4
  x = con_block(x,3,[512,512,2048],s=2)
  x = identity_block(x,3,[512,512,2048])
  x = identity_block(x,3,[512,512,2048])

  # AVGPOOL 
  x = AveragePooling2D(pool_size=(2, 2), padding='same')(x)

 # output layer
  x = Flatten()(x)
  x = Dense(output_classes, activation='softmax', name='fc' + str(output_classes), 
              kernel_initializer=glorot_uniform(seed=0))(x)

  # Create model
  model = Model(inputs=x_input, outputs=x, name='ResNet101')

  return model

  
                     

                  